In [1]:
from utils import *
from ucb import main, interact, trace
from datetime import datetime

In [2]:
def choose(paragraphs, select, k):
    """Return the Kth paragraph from PARAGRAPHS for which SELECT called on the
    paragraph returns true. If there are fewer than K such paragraphs, return
    the empty string.
    """
    # BEGIN PROBLEM 1
    count = 0
    for para in paragraphs:
        if select(para) and count == k:            
            return para
        elif select(para):
            count += 1
    return ""    
    # END PROBLEM 1

In [3]:
def about(topic):
    """Return a select function that returns whether a paragraph contains one
    of the words in TOPIC.

    >>> about_dogs = about(['dog', 'dogs', 'pup', 'puppy'])
    >>> choose(['Cute Dog!', 'That is a cat.', 'Nice pup!'], about_dogs, 0)
    'Cute Dog!'
    >>> choose(['Cute Dog!', 'That is a cat.', 'Nice pup.'], about_dogs, 1)
    'Nice pup.'
    """
    assert all([lower(x) == x for x in topic]), 'topics should be lowercase.'
    # BEGIN PROBLEM 2
    def select(para):
        test = split(lower(remove_punctuation(para)))
        for word in topic:
            if word in test:
                return True
        return False
    return select
    # END PROBLEM 2

In [4]:
def accuracy(typed, reference):
    """Return the accuracy (percentage of words typed correctly) of TYPED
    when compared to the prefix of REFERENCE that was typed.

    >>> accuracy('Cute Dog!', 'Cute Dog.')
    50.0
    >>> accuracy('A Cute Dog!', 'Cute Dog.')
    0.0
    >>> accuracy('cute Dog.', 'Cute Dog.')
    50.0
    >>> accuracy('Cute Dog. I say!', 'Cute Dog.')
    50.0
    >>> accuracy('Cute', 'Cute Dog.')
    100.0
    >>> accuracy('', 'Cute Dog.')
    0.0
    """
    typed_words = split(typed)
    reference_words = split(reference)
    # BEGIN PROBLEM 3
    right = 0
    wrong = 0
    for idx in range(len(typed_words)):
        if idx < len(reference_words):
            if typed_words[idx] == reference_words[idx]:
                right += 1
            else:
                wrong += 1
        else:
            wrong += 1
    if len(typed_words) == 0:
        return 0.0
    else:
        return 100.0 * right / len(typed_words)
    # END PROBLEM 3

In [5]:
def wpm(typed, elapsed):
    """Return the words-per-minute (WPM) of the TYPED string."""
    assert elapsed > 0, 'Elapsed time must be positive'
    # BEGIN PROBLEM 4
    return (len(typed) / 5) / (elapsed / 60)
    # END PROBLEM 4

In [6]:
def autocorrect(user_word, valid_words, diff_function, limit):
    """Returns the element of VALID_WORDS that has the smallest difference
    from USER_WORD. Instead returns USER_WORD if that difference is greater
    than LIMIT.
    """
    # BEGIN PROBLEM 5
    if user_word in valid_words:
        return user_word
    else:
        min_diff = float("inf")
        best_word = user_word
        for word in valid_words:
            diff = diff_function(user_word, word, limit)
            if diff < min_diff and diff <= limit:
                min_diff = diff
                best_word = word
        return best_word
    # END PROBLEM 5

In [7]:
def sphinx_swap(start, goal, limit):
    """A diff function for autocorrect that determines how many letters
    in START need to be substituted to create GOAL, then adds the difference in
    their lengths. 
    """
    # BEGIN PROBLEM 6
#    assert False, 'Remove this line'
    if limit == 0:
        if start == goal:
            return 0
        else:
            return 1
    elif len(start) == 0:
        return min(len(goal), limit+1)
    elif len(goal) == 0:
        return min(len(start), limit+1)
    if start[0] == goal[0]:
        return sphinx_swap(start[1:], goal[1:], limit)
    else:
        return sphinx_swap(start[1:], goal[1:], limit-1) + 1
    # END PROBLEM 6

In [8]:
def feline_fixes(start, goal, limit):
    """A diff function that computes the edit distance from START to GOAL."""
#    assert False, 'Remove this line'

    if limit == 0:
        if start == goal:
            return 0
        else:
            return 1
    elif len(start) == 0:
        return min(len(goal), limit+1)
    elif len(goal) == 0:
        return min(len(start), limit+1)
    
    else:
        add_diff = feline_fixes(start, goal[1:], limit-1)
        remove_diff = feline_fixes(start[1:], goal, limit-1)
        substitute_diff = feline_fixes(start[1:], goal[1:], limit-1)
        if start[0] == goal[0]:
            return feline_fixes(start[1:], goal[1:], limit)
        else:
            return min(add_diff, remove_diff, substitute_diff) + 1

In [9]:
def report_progress(typed, prompt, id, send):
    """Send a report of your id and progress so far to the multiplayer server."""
    # BEGIN PROBLEM 8
    count = 0
    for idx in range(len(typed)):
        if typed[idx] == prompt[idx]:
            count += 1
        else:
            break
    progress = count/len(prompt)
    send({'id': id, 'progress': progress})
    return progress
    # END PROBLEM 8

In [10]:
def fastest_words_report(times_per_player, words):
    """Return a text description of the fastest words typed by each player."""
    game = time_per_word(times_per_player, words)
    fastest = fastest_words(game)
    report = ''
    for i in range(len(fastest)):
        words = ','.join(fastest[i])
        report += 'Player {} typed these fastest: {}\n'.format(i + 1, words)
    return report

In [11]:
def game(words, times):
    """A data abstraction containing all words typed and their times."""
    assert all([type(w) == str for w in words]), 'words should be a list of strings'
    assert all([type(t) == list for t in times]), 'times should be a list of lists'
    assert all([isinstance(i, (int, float)) for t in times for i in t]), 'times lists should contain numbers'
    assert all([len(t) == len(words) for t in times]), 'There should be one word per time.'
    return [words, times]


def word_at(game, word_index):
    """A selector function that gets the word with index word_index"""
    assert 0 <= word_index < len(game[0]), "word_index out of range of words"
    return game[0][word_index]


def all_words(game):
    """A selector function for all the words in the game"""
    return game[0]


def all_times(game):
    """A selector function for all typing times for all players"""
    return game[1]


def time(game, player_num, word_index):
    """A selector function for the time it took player_num to type the word at word_index"""
    assert word_index < len(game[0]), "word_index out of range of words"
    assert player_num < len(game[1]), "player_num out of range of players"
    return game[1][player_num][word_index]


def game_string(game):
    """A helper function that takes in a game object and returns a string representation of it"""
    return "game(%s, %s)" % (game[0], game[1])

In [12]:
def time_per_word(times_per_player, words):
    """Given timing data, return a game data abstraction, which contains a list
    of words and the amount of time each player took to type each word.

    Arguments:
        times_per_player: A list of lists of timestamps including the time
                          the player started typing, followed by the time
                          the player finished typing each word.
        words: a list of words, in the order they are typed.
    """
    # BEGIN PROBLEM 9
    times = [[lst[idx+1] - lst[idx] for idx in range(len(lst)-1)] for lst in times_per_player]
    return game(words, times)
    # END PROBLEM 9

In [21]:
def fastest_words(game):
    """Return a list of lists of which words each player typed fastest.

    Arguments:
        game: a game data abstraction as returned by time_per_word.
    Returns:
        a list of lists containing which words each player typed fastest
    """
    players = range(len(all_times(game)))  # An index for each player
    words = range(len(all_words(game)))    # An index for each word
    # BEGIN PROBLEM 10
    fast = [[] for _ in players]
    for idx in words:
        fast_time = float("inf")
        fast_pl = float("inf")
        for player in players:
            p_time = time(game, player, idx)
            if p_time < fast_time:
                fast_time = p_time
                fast_pl = player
        fast[fast_pl].append(word_at(game, idx))
    return fast    
    # END PROBLEM 10